In [1]:
%autosave 0

Autosave disabled


In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier as rfc
from adam_prepare import titanic_pipeline

seed = 42

Let's read in our data from the titanic pipeline function!

In [2]:
train, val, test = titanic_pipeline()
train.shape, val.shape, test.shape

((623, 9), (134, 9), (134, 9))

We need to create our X and y subsets.

In [6]:
x_train = train.drop(columns = 'survived')
y_train = train.survived
x_val = val.drop(columns = 'survived')
y_val = val.survived

In [12]:
x_train = pd.get_dummies(x_train)
x_val = pd.get_dummies(x_val)

,age,sibsp,parch,fare,alone,sex_female,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,19.0,1,0,53.1000,0,False,True,True,False,False,False,False,True
45,29.0,0,0,8.0500,1,False,True,False,False,True,False,False,True
28,29.0,0,0,7.8792,1,True,False,False,False,True,False,True,False
633,29.0,0,0,0.0000,1,False,True,True,False,False,False,False,True
403,28.0,1,0,15.8500,0,False,True,False,False,True,False,False,True


Before modeling, it's always important to define a baseline!

In [7]:
y_train.value_counts()b

survived
0    384
1    239
Name: count, dtype: int64

In [8]:
(y_train == 0).mean()

0.6163723916532905

The majority of people died on the Titanic. Guessing died every time has 62% accuracy.

Now we are ready to create a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) object and start modeling!

In [13]:
rf = rfc(max_depth = 5, random_state = seed)

rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [16]:
rf.score(x_train, y_train)

0.8507223113964687

In [17]:
rf.score(x_val, y_val)

0.8582089552238806

In [18]:
rf.feature_importances_

array([0.09672174, 0.03190949, 0.03734122, 0.12762573, 0.01634038,
       0.25532888, 0.26181189, 0.04777106, 0.0195783 , 0.07229473,
       0.01166075, 0.00479926, 0.01681657])

In [21]:
fi = pd.DataFrame({'feature': x_train.columns,
             'importance': rf.feature_importances_})
fi.sort_values(by = 'importance' , ascending = False)

,feature,importance
6,sex_male,0.261812
5,sex_female,0.255329
3,fare,0.127626
0,age,0.096722
9,class_Third,0.072295
7,class_First,0.047771
2,parch,0.037341
1,sibsp,0.031909
8,class_Second,0.019578
12,embark_town_Southampton,0.016817


In [22]:
# Lets make another that is just better?
rf_big = rfc(max_depth = 5, random_state = seed, n_estimators=2197)

rf_big.fit(x_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=2197, random_state=42)

In [23]:
big_fi = pd.DataFrame({'feature': x_train.columns,
             'importance': rf_big.feature_importances_})
big_fi.sort_values(by = 'importance' , ascending = False)

,feature,importance
5,sex_female,0.255388
6,sex_male,0.249429
3,fare,0.132774
0,age,0.102349
9,class_Third,0.074339
7,class_First,0.049488
1,sibsp,0.032486
2,parch,0.031726
4,alone,0.019122
8,class_Second,0.018359


In [24]:
rf_big.score(x_train, y_train)


0.8523274478330658

In [25]:
rf_big.score(x_val, y_val)

0.8582089552238806